In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-Embedding-8B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-Embedding-8B")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of Qwen3ForCausalLM were not initialized from the model checkpoint at Qwen/Qwen3-Embedding-8B and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [2]:
import torch
import torch.nn.functional as F

def classify_email_qwen3(email_body):
    # 1. Define instructions and labels
    # The 'Instruct' prefix is essential for aligning the embedding space correctly
    instruction = "Instruct: Classify this email for security purposes. Is it a fraudulent phishing attempt or a legitimate business communication?\nQuery: "
    
    labels = [
        "A malicious phishing email containing urgent threats, suspicious links, or credential harvesting.",
        "A safe, legitimate, and professional business email communication."
    ]
    
    # 2. Combine inputs [Email, Phishing Label, Legitimate Label]
    all_texts = [f"{instruction}{email_body}"] + labels
    
    # 3. Tokenize
    inputs = tokenizer(all_texts, padding=True, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        
        # Get the last layer of hidden states
        last_hidden_states = outputs.hidden_states[-1]
        
        # Extract the [EOS] token embedding (the last non-padding token)
        last_token_indices = inputs.attention_mask.sum(dim=1) - 1
        embeddings = last_hidden_states[torch.arange(last_hidden_states.size(0)), last_token_indices]
        
        # Normalize for Cosine Similarity
        embeddings = F.normalize(embeddings, p=2, dim=1)

    # 4. Compare the Email vector (index 0) to the Label vectors (index 1 and 2)
    email_vec = embeddings[0:1]
    label_vecs = embeddings[1:]

    # Apply a 'Temperature' scale of 20.0 to widen the gap between scores
    logits = torch.matmul(email_vec, label_vecs.T) * 20.0
    probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

    return {
        "Phishing": probs[0],
        "Legitimate": probs[1],
        "Result": "Phishing" if probs[0] > probs[1] else "Legitimate"
    }

In [5]:
test_email = """
Subject: Rescheduling: Project Alpha Weekly Standup

Body: Hi Team, I have a conflict during our usual 10:00 AM slot tomorrow. I've moved the meeting to 1:30 PM. We will use the same Microsoft Teams link. I've updated the calendar invite accordingly. See you then!
"""

result = classify_email_qwen3(test_email)

print(f"Decision: {result['Result']}")
print(f"Phishing Confidence: {result['Phishing']:.2%}")
print(f"Legit Confidence: {result['Legitimate']:.2%}")

Decision: Legitimate
Phishing Confidence: 0.02%
Legit Confidence: 99.98%
